In [ ]:
import sys
import csv
import os.path
import requests
import shutil
import xml.etree.ElementTree as ET
from PIL import Image
from urllib.parse import urlparse

In [ ]:
class Tile:
    def __init__(self):
        self.path = ''

    def __str__(self):
        return(self.path)

    def __repr__(self):
        return str(self)

class Column:
    def __init__(self):
        self.tiles = []

    def __str__(self):
        return(str(self.tiles))

    def __repr__(self):
        return str(self)

class Level:
    def __init__(self, path_template, level, width, height):
        self.tile_size = 512
        self.level = level
        self.width = int(width)
        self.height = int(height)
        self.num_columns = (self.width + self.tile_size - 1) // self.tile_size
        self.num_tiles = (self.height + self.tile_size - 1) // self.tile_size
        self.path = path_template.replace('%c', '0').replace('%l', str(level)).replace('%x', '0').replace('%y', '0')
        self.path = os.path.split(os.path.split(self.path)[0])[0]

        # http://www.xrez.com/wp-content/uploads/2019/07/ElCap800_100k_final_routes/tiles/node1/cf_0/l_2/c_3/tile_0.jpg

        self.columns = []
        for c in range(self.num_columns):
            col = Column()
            col.path = path_template.replace('%c', '0').replace('%l', str(level)).replace('%x', str(c)).replace('%y', '0')
            col.path = os.path.split(col.path)[0]
            for t in range(self.num_tiles):
                tile = Tile()
                tile.path = path_template.replace('%c', '0').replace('%l', str(level)).replace('%x', str(c)).replace('%y', str(t))
                # tiles/node1/cf_%c/l_%l/c_%x/tile_%y.jpg
                col.tiles.append(tile) 
            self.columns.append(col)

    def __str__(self):
        return("Level " + str(self.level) + ": " + str(self.width) + "x" + str(self.height) + "(" + str(self.num_columns) + "," + str(self.num_tiles) + ")\n" + str(self.columns))

    def __repr__(self):
        return str(self)

# pano url
url = 'http://www.xrez.com/wp-content/uploads/2019/07/ElCap800_100k_final_routes/pano.xml'
resp = requests.get(url)
root = ET.fromstring(resp.content)


# tiles/node1/cf_%c/l_%l/c_%x/tile_%y.jpg
tile_path_template = root.find('./input').attrib['leveltileurl']
parsed_url = urlparse(url)
# wp-content/uploads/2019/07/ElCap800_100k_final_routes
base_path = os.path.split(parsed_url.path)[0]
# http://www.xrez.com/wp-content/uploads/2019/07/ElCap800_100k_final_routes
base_url = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_url) + base_path 

levels = []
current_level = 0
for item in reversed(root.findall('./input/level')):
    l = Level(tile_path_template, current_level, item.attrib['width'], item.attrib['height'])
    levels.append(l)
    current_level = current_level + 1

In [ ]:
# Download all tiles
tiles = []
for l in levels:
    for c in l.columns:
        for t in c.tiles:
            tiles.append(t.path)

for tile in tiles:
    image_url = base_url + '/' + tile
    
    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True
        
        # Open a local file with wb ( write binary ) permission.
        os.makedirs(os.path.split(tile)[0], exist_ok=True)
        with open(tile,'wb') as f:
            shutil.copyfileobj(r.raw, f)
            
        # print('Image sucessfully Downloaded: ', tile)
    else:
        print('Image Couldn\'t be retreived: ', image_url)

In [ ]:
import cv2
import numpy as np

for l in levels:
    l_path = l.path + '/concat.jpg'
    if os.path.exists(l_path):
        continue
    
    print('Processing level ', str(l.level))
    level_images = []
    for c in l.columns: 
        c_path = c.path + '/concat.jpg'
        if os.path.exists(c_path):
            im = cv2.imread(c_path)
        else:
            col_images = [] 
            for t in c.tiles:
                col_images.append(cv2.imread(t.path))
            im = cv2.vconcat(col_images)
            cv2.imwrite(c_path, im)
        level_images.append(im)
        
    print('Concatenating images for level ', str(l.level))
    l_path = l.path + '/concat_0.jpg'
    im = cv2.hconcat(level_images[:98])
    print('Writing image ', l_path)
    cv2.imwrite(l_path, im)

    l_path = l.path + '/concat_1.jpg'
    im = cv2.hconcat(level_images[98:])
    print('Writing image ', l_path)
    cv2.imwrite(l_path, im)